# RNN and LSTM Train
This notebook will train an RNN and LSTM models on the Patent Classification and Bankings77 Dataset for different regularization values. It will load in the different raw data files and preprocess the data and then train each model for the regularization parameters.

In [1]:
import json
import nltk
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import tensorflow as tf
from dropconnect_tensorflow import DropConnect
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.regularizers import l1, l2

## Initialization

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ap4136/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
seed = 22
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.keras.utils.set_random_seed(seed)

In [4]:
def load_patent_data(data_path):
    train = [json.loads(line) for line in open(f'{data_path}/train_data.txt', 'rb')]
    val = [json.loads(line) for line in open(f'{data_path}/val_data.txt', 'rb')]
    test = [json.loads(line) for line in open(f'{data_path}/test_data.txt', 'rb')]
    data = train + val + test
    text = [d['abstract'] for d in data]
    labels = [d['label'] for d in data]
    temp = list(zip(text, labels))
    random.shuffle(temp)
    text, labels = zip(*temp)
    return text[:20000], labels[:20000]

In [5]:
def load_bankings_data(data_path):
    train = pd.read_csv(f'{data_path}/train.csv')
    test = pd.read_csv(f'{data_path}/test.csv')
    data = pd.concat([train, test])
    texts = data['text'].tolist()
    labels = data['category'].tolist()
    temp = list(zip(texts, labels))
    random.shuffle(temp)
    texts, labels = zip(*temp)
    texts, labels = list(texts), list(labels)
    return texts[:8000], labels[:8000]

In [6]:
def load_article_data(data_path):
    texts, labels = [], []
    data = pd.read_csv(f'{data_path}/bbc-text.csv')
    for row in data.iterrows():
        texts.append(row[1].text)
        labels.append(row[1].category)
    return texts, labels

In [7]:
def load_data(data_path):
    if os.path.basename(data_path) == 'patent_classification':
        texts, labels = load_patent_data(data_path)
    elif os.path.basename(data_path) == 'bankings_77':
        texts, labels = load_bankings_data(data_path)
    elif os.path.basename(data_path) == 'article_classification':
        texts, labels = load_article_data(data_path)
    return texts, labels

In [8]:
def get_preprocessed_data(texts, labels):
    special_symbols = re.compile('[/(){}\[\]\|@,;]')
    bad_symbols = re.compile('[^0-9a-z #+_]')
    stopwords_set = set(stopwords.words('english'))
    processed_texts = []
    for i, text in enumerate(texts):
        processed_text = text.lower()
        processed_text = special_symbols.sub(' ', processed_text)
        processed_text = bad_symbols.sub('', processed_text)
        processed_text = ' '.join(word for word in processed_text.split() if word not in stopwords_set)
        processed_texts.append(processed_text)
    return processed_texts, labels

In [9]:
def split_data(texts, labels, validation_size, test_size):
    sequences, test_sequences, labels, test_labels = train_test_split(texts, labels, test_size=test_size, shuffle=False)
    train_sequences, validation_sequences, train_labels, validation_labels = train_test_split(sequences, labels, test_size=validation_size, shuffle=False)
    return [train_sequences, validation_sequences, test_sequences], [train_labels, validation_labels, test_labels]

In [10]:
def tokenize_data(X, y, labels, vocab_size, oov_token, padding_size, padding_type):
    sequence_tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
    sequence_tokenizer.fit_on_texts(X[0])
    X_train = sequence_tokenizer.texts_to_sequences(X[0])
    X_train = pad_sequences(X_train, maxlen=padding_size, padding=padding_type)
    X_validation = sequence_tokenizer.texts_to_sequences(X[1])
    X_validation = pad_sequences(X_validation, maxlen=padding_size, padding=padding_type)
    X_test = sequence_tokenizer.texts_to_sequences(X[2])
    X_test = pad_sequences(X_test, maxlen=padding_size, padding=padding_type)
    label_tokenizer = LabelEncoder()
    label_tokenizer.fit(labels)
    y_train = label_tokenizer.transform(y[0])
    y_validation = label_tokenizer.transform(y[1])
    y_test = label_tokenizer.transform(y[2])
    return [X_train, X_validation, X_test], [y_train, y_validation, y_test]

In [11]:
def get_model(vocab_size, hidden_layer_size, activation, num_labels, weight_decay, dropout_rate, dropconnect_rate, model_type):
    if model_type == 'rnn':
        sequential_layer = tf.keras.layers.SimpleRNN(hidden_layer_size, activation, kernel_regularizer=weight_decay, dropout=dropout_rate)
    elif model_type == 'lstm':
        sequential_layer = tf.keras.layers.LSTM(hidden_layer_size, activation, kernel_regularizer=weight_decay, dropout=dropout_rate)
    sequential_layer = tf.keras.layers.Bidirectional(sequential_layer)
    if dropconnect_rate > 0:
        sequential_layer = DropConnect(sequential_layer, prob=dropconnect_rate)
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, hidden_layer_size),
        sequential_layer,
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
        tf.keras.layers.Dense(num_labels, activation='softmax')
    ])
    return model

In [12]:
def run_model(X, y, model, loss, optimizer, metrics, epochs, batch_size, save_path, model_name):
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    history = model.fit(X[0], y[0], batch_size=batch_size, epochs=epochs, validation_data=(X[1], y[1]))
    model.evaluate(X[2], y[2], batch_size=batch_size)
    model.save(f'{save_path}/{model_name}.h5')
    with open(f'{save_path}/{model_name}.pickle', 'wb') as f:
        pickle.dump(history.history, f)

In [13]:
save_path = './Saved Models'
validation_size = 0.15
test_size = 0.2
vocab_size = 5000
oov_token = '<OOV>'
padding_size = 200
padding_type = 'post'

## Patent Classification Dataset Training

In [14]:
data_path = './Data/patent_classification'
texts, labels = load_data(data_path)
processed_text, labels = get_preprocessed_data(texts, labels)
X, y = split_data(processed_text, labels, validation_size, test_size)
X, y = tokenize_data(X, y, labels, vocab_size, oov_token, padding_size, padding_type)

In [16]:
hidden_layer_size = 128
num_labels = len(set(labels))
loss = 'sparse_categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy']
epochs = 10
batch_size = 128
weight_decays_l1 = [0, 0.001, 0.01, 0.1]
weight_decays_l2 = [0.001, 0.01, 0.1]
dropout_rates = [0.1, 0.2, 0.3]
dropconnect_rates = [0.1, 0.2, 0.3]
dataset = os.path.basename(data_path)

In [17]:
for weight_decay_l1 in weight_decays_l1:
    print(f'Training RNN on {dataset} with {weight_decay_l1} L1 Weight Decay...')
    weight_decay = l1(weight_decay_l1) if weight_decay_l1 > 0 else None  
    rnn_model = get_model(vocab_size, hidden_layer_size, 'relu', num_labels, weight_decay, 0, 0, 'rnn')
    run_model(X, y, rnn_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'rnn_l1_weight_decay_{weight_decay_l1}_{dataset}')

Training RNN on patent_classification with 0 L1 Weight Decay...


2022-05-07 04:04:06.430151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 04:04:06.442191: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 04:04:06.442860: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 04:04:06.444071: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/10
107/107 [==============================] - 37s 328ms/step - loss: 1.8967 - accuracy: 0.2507 - val_loss: 1.6623 - val_accuracy: 0.3033
Epoch 2/10
107/107 [==============================] - 35s 332ms/step - loss: 1.5029 - accuracy: 0.4179 - val_loss: 1.5031 - val_accuracy: 0.4412
Epoch 3/10
107/107 [==============================] - 35s 324ms/step - loss: 1.2212 - accuracy: 0.5646 - val_loss: 1.4026 - val_accuracy: 0.5021
Epoch 4/10
107/107 [==============================] - 35s 324ms/step - loss: 1.0153 - accuracy: 0.6524 - val_loss: 1.4229 - val_accuracy: 0.5304
Epoch 5/10
107/107 [==============================] - 35s 329ms/step - loss: 3.5894 - accuracy: 0.7130 - val_loss: 1.6099 - val_accuracy: 0.5129
Epoch 6/10
107/107 [==============================] - 34s 316ms/step - loss: 0.6908 - accuracy: 0.7760 - val_loss: 1.6913 - val_accuracy: 0.5188
Epoch 7/10
107/107 [==============================] - 35s 325ms/step - loss: 0.5592 - accuracy: 0.8196 - val_loss: 1.7124 - val_ac

In [18]:
for weight_decay_l2 in weight_decays_l2:
    print(f'Training RNN on {dataset} with {weight_decay_l2} L2 Weight Decay...')
    weight_decay = l2(weight_decay_l2) if weight_decay_l2 > 0 else None
    rnn_model = get_model(vocab_size, hidden_layer_size, 'relu', num_labels, weight_decay, 0, 0, 'rnn')
    run_model(X, y, rnn_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'rnn_l2_weight_decay_{weight_decay_l2}_{dataset}')

Training RNN on patent_classification with 0.001 L2 Weight Decay...
Epoch 1/10
107/107 [==============================] - 34s 311ms/step - loss: 1.9490 - accuracy: 0.2889 - val_loss: 1.7338 - val_accuracy: 0.3558
Epoch 2/10
107/107 [==============================] - 32s 300ms/step - loss: 1.5599 - accuracy: 0.4238 - val_loss: 1.6180 - val_accuracy: 0.4112
Epoch 3/10
107/107 [==============================] - 33s 308ms/step - loss: 1.3742 - accuracy: 0.5060 - val_loss: 1.5413 - val_accuracy: 0.4508
Epoch 4/10
107/107 [==============================] - 33s 311ms/step - loss: 1.1812 - accuracy: 0.6104 - val_loss: 1.5329 - val_accuracy: 0.5146
Epoch 5/10
107/107 [==============================] - 32s 295ms/step - loss: 1.0013 - accuracy: 0.6875 - val_loss: 1.5175 - val_accuracy: 0.5271
Epoch 6/10
107/107 [==============================] - 33s 305ms/step - loss: 0.8335 - accuracy: 0.7443 - val_loss: 1.5664 - val_accuracy: 0.5117
Epoch 7/10
107/107 [==============================] - 32s 304m

In [19]:
for dropout_rate in dropout_rates:
    print(f'Training RNN on {dataset} with {dropout_rate} Dropout...')
    rnn_model = get_model(vocab_size, hidden_layer_size, 'relu', num_labels, None, dropout_rate, 0, 'rnn')
    run_model(X, y, rnn_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'rnn_dropout_{dropout_rate}_{dataset}')

Training RNN on patent_classification with 0.1 Dropout...
Epoch 1/10
107/107 [==============================] - 45s 411ms/step - loss: 1.8542 - accuracy: 0.2682 - val_loss: 1.5740 - val_accuracy: 0.3750
Epoch 2/10
107/107 [==============================] - 43s 401ms/step - loss: 1.4472 - accuracy: 0.4324 - val_loss: 1.4842 - val_accuracy: 0.4258
Epoch 3/10
107/107 [==============================] - 44s 408ms/step - loss: 1.2470 - accuracy: 0.5413 - val_loss: 1.4663 - val_accuracy: 0.4588
Epoch 4/10
107/107 [==============================] - 45s 417ms/step - loss: 1.0596 - accuracy: 0.6352 - val_loss: 1.4134 - val_accuracy: 0.5387
Epoch 5/10
107/107 [==============================] - 42s 396ms/step - loss: 1.1592 - accuracy: 0.6896 - val_loss: 1.4840 - val_accuracy: 0.5100
Epoch 6/10
107/107 [==============================] - 43s 405ms/step - loss: 0.8029 - accuracy: 0.7355 - val_loss: 1.5897 - val_accuracy: 0.5146
Epoch 7/10
107/107 [==============================] - 43s 404ms/step - l

In [21]:
rnn_model = get_model(vocab_size, hidden_layer_size, 'relu', num_labels, None, 0.2, 0, 'rnn')
run_model(X, y, rnn_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'rnn_dropout_0.2_patent_classification')

Epoch 1/10
107/107 [==============================] - 45s 409ms/step - loss: 1.8414 - accuracy: 0.2616 - val_loss: 1.6518 - val_accuracy: 0.3438
Epoch 2/10
107/107 [==============================] - 45s 418ms/step - loss: 1.5145 - accuracy: 0.4115 - val_loss: 1.5429 - val_accuracy: 0.4242
Epoch 3/10
107/107 [==============================] - 44s 410ms/step - loss: 1.2862 - accuracy: 0.5411 - val_loss: 1.4305 - val_accuracy: 0.4938
Epoch 4/10
107/107 [==============================] - 43s 402ms/step - loss: 82.1206 - accuracy: 0.5833 - val_loss: 1.4894 - val_accuracy: 0.5117
Epoch 5/10
107/107 [==============================] - 42s 391ms/step - loss: 0.9672 - accuracy: 0.6779 - val_loss: 1.4420 - val_accuracy: 0.5267
Epoch 6/10
107/107 [==============================] - 43s 404ms/step - loss: 0.8294 - accuracy: 0.7243 - val_loss: 1.5779 - val_accuracy: 0.5083
Epoch 7/10
107/107 [==============================] - 44s 408ms/step - loss: 0.7492 - accuracy: 0.7560 - val_loss: 1.5363 - val_a

In [22]:
for dropconnect_rate in dropconnect_rates:
    print(f'Training RNN on {dataset} with {dropconnect_rate} Dropconnect...')
    rnn_model = get_model(vocab_size, hidden_layer_size, 'relu', num_labels, None, 0, dropconnect_rate, 'rnn')
    run_model(X, y, rnn_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'rnn_dropconnect_{dropconnect_rate}_{dataset}')

Training RNN on patent_classification with 0.1 Dropconnect...
Epoch 1/10
107/107 [==============================] - 36s 320ms/step - loss: 1.7508 - accuracy: 0.2901 - val_loss: 1.6292 - val_accuracy: 0.3529
Epoch 2/10
107/107 [==============================] - 33s 306ms/step - loss: 1.4559 - accuracy: 0.4254 - val_loss: 1.5178 - val_accuracy: 0.4288
Epoch 3/10
107/107 [==============================] - 34s 318ms/step - loss: 1.2385 - accuracy: 0.5563 - val_loss: 1.3874 - val_accuracy: 0.5163
Epoch 4/10
107/107 [==============================] - 33s 305ms/step - loss: 1.0038 - accuracy: 0.6585 - val_loss: 1.4210 - val_accuracy: 0.5471
Epoch 5/10
107/107 [==============================] - 34s 316ms/step - loss: 0.8213 - accuracy: 0.7276 - val_loss: 1.5098 - val_accuracy: 0.5079
Epoch 6/10
107/107 [==============================] - 34s 319ms/step - loss: 0.6796 - accuracy: 0.7792 - val_loss: 1.5482 - val_accuracy: 0.5154
Epoch 7/10
107/107 [==============================] - 33s 309ms/step

In [16]:
hidden_layer_size = 128
num_labels = len(set(labels))
loss = 'sparse_categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy']
epochs = 10
batch_size = 64
weight_decays_l1 = [0, 0.001, 0.01, 0.1]
weight_decays_l2 = [0.001, 0.01, 0.1]
dropout_rates = [0.1, 0.2, 0.3]
dropconnect_rates = [0.1, 0.2, 0.3]
dataset = os.path.basename(data_path)

In [17]:
for weight_decay_l1 in weight_decays_l1:
    print(f'Training LSTM on {dataset} with {weight_decay_l1} L1 Weight Decay...')
    weight_decay = l1(weight_decay_l1) if weight_decay_l1 > 0 else None
    lstm_model = get_model(vocab_size, hidden_layer_size, 'tanh', num_labels, weight_decay, 0, 0, 'lstm')
    run_model(X, y, lstm_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'lstm_l1_weight_decay_{weight_decay_l1}_{dataset}')

Training LSTM on patent_classification with 0 L1 Weight Decay...


2022-05-07 03:21:10.600007: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 03:21:10.613074: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 03:21:10.613797: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 03:21:10.615050: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/10


2022-05-07 03:21:14.596071: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


213/213 [==============================] - 12s 42ms/step - loss: 1.6080 - accuracy: 0.3890 - val_loss: 1.3953 - val_accuracy: 0.5000
Epoch 2/10
213/213 [==============================] - 8s 38ms/step - loss: 1.1859 - accuracy: 0.5906 - val_loss: 1.2741 - val_accuracy: 0.5621
Epoch 3/10
213/213 [==============================] - 8s 38ms/step - loss: 0.9842 - accuracy: 0.6657 - val_loss: 1.3442 - val_accuracy: 0.5437
Epoch 4/10
213/213 [==============================] - 8s 38ms/step - loss: 0.8354 - accuracy: 0.7161 - val_loss: 1.4060 - val_accuracy: 0.5437
Epoch 5/10
213/213 [==============================] - 8s 38ms/step - loss: 0.7147 - accuracy: 0.7576 - val_loss: 1.4948 - val_accuracy: 0.5417
Epoch 6/10
213/213 [==============================] - 8s 39ms/step - loss: 0.6038 - accuracy: 0.8016 - val_loss: 1.5761 - val_accuracy: 0.5383
Epoch 7/10
213/213 [==============================] - 8s 39ms/step - loss: 0.5024 - accuracy: 0.8387 - val_loss: 1.7668 - val_accuracy: 0.5129
Epoch 8/1

In [18]:
for weight_decay_l2 in weight_decays_l2:
    print(f'Training LSTM on {dataset} with {weight_decay_l2} L2 Weight Decay...')
    weight_decay = l2(weight_decay_l2) if weight_decay_l2 > 0 else None
    lstm_model = get_model(vocab_size, hidden_layer_size, 'tanh', num_labels, weight_decay, 0, 0, 'lstm')
    run_model(X, y, lstm_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'lstm_l2_weight_decay_{weight_decay_l2}_{dataset}')

Training LSTM on patent_classification with 0.001 L2 Weight Decay...
Epoch 1/10
213/213 [==============================] - 12s 43ms/step - loss: 1.7412 - accuracy: 0.3468 - val_loss: 1.5228 - val_accuracy: 0.4250
Epoch 2/10
213/213 [==============================] - 8s 39ms/step - loss: 1.2969 - accuracy: 0.5521 - val_loss: 1.3551 - val_accuracy: 0.5483
Epoch 3/10
213/213 [==============================] - 8s 38ms/step - loss: 1.0789 - accuracy: 0.6392 - val_loss: 1.3810 - val_accuracy: 0.5300
Epoch 4/10
213/213 [==============================] - 8s 38ms/step - loss: 0.9465 - accuracy: 0.6869 - val_loss: 1.4881 - val_accuracy: 0.5446
Epoch 5/10
213/213 [==============================] - 8s 38ms/step - loss: 0.8352 - accuracy: 0.7249 - val_loss: 1.4637 - val_accuracy: 0.5471
Epoch 6/10
213/213 [==============================] - 8s 38ms/step - loss: 0.7233 - accuracy: 0.7688 - val_loss: 1.5705 - val_accuracy: 0.5375
Epoch 7/10
213/213 [==============================] - 8s 38ms/step - los

In [19]:
for dropout_rate in dropout_rates:
    print(f'Training LSTM on {dataset} with {dropout_rate} Dropout...')
    lstm_model = get_model(vocab_size, hidden_layer_size, 'tanh', num_labels, None, dropout_rate, 0, 'lstm')
    run_model(X, y, lstm_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'lstm_dropout_{dropout_rate}_{dataset}')

Training LSTM on patent_classification with 0.1 Dropout...
Epoch 1/10
213/213 [==============================] - 11s 42ms/step - loss: 1.6479 - accuracy: 0.3749 - val_loss: 1.4022 - val_accuracy: 0.5071
Epoch 2/10
213/213 [==============================] - 8s 39ms/step - loss: 1.2121 - accuracy: 0.5801 - val_loss: 1.2944 - val_accuracy: 0.5675
Epoch 3/10
213/213 [==============================] - 8s 39ms/step - loss: 1.0063 - accuracy: 0.6574 - val_loss: 1.3146 - val_accuracy: 0.5558
Epoch 4/10
213/213 [==============================] - 8s 39ms/step - loss: 0.8542 - accuracy: 0.7135 - val_loss: 1.3821 - val_accuracy: 0.5554
Epoch 5/10
213/213 [==============================] - 9s 40ms/step - loss: 0.7262 - accuracy: 0.7620 - val_loss: 1.4774 - val_accuracy: 0.5487
Epoch 6/10
213/213 [==============================] - 8s 39ms/step - loss: 0.6220 - accuracy: 0.7976 - val_loss: 1.5525 - val_accuracy: 0.5396
Epoch 7/10
213/213 [==============================] - 8s 39ms/step - loss: 0.5302 

In [20]:
for dropconnect_rate in dropconnect_rates:
    print(f'Training LSTM on {dataset} with {dropconnect_rate} Dropconnect...')
    lstm_model = get_model(vocab_size, hidden_layer_size, 'tanh', num_labels, None, 0, dropconnect_rate, 'lstm')
    run_model(X, y, lstm_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'lstm_dropconnect_{dropconnect_rate}_{dataset}')

Training LSTM on patent_classification with 0.1 Dropconnect...
Epoch 1/10
213/213 [==============================] - 12s 43ms/step - loss: 1.6339 - accuracy: 0.3547 - val_loss: 1.4247 - val_accuracy: 0.4888
Epoch 2/10
213/213 [==============================] - 8s 39ms/step - loss: 1.2109 - accuracy: 0.5777 - val_loss: 1.2905 - val_accuracy: 0.5575
Epoch 3/10
213/213 [==============================] - 8s 39ms/step - loss: 0.9965 - accuracy: 0.6585 - val_loss: 1.3319 - val_accuracy: 0.5487
Epoch 4/10
213/213 [==============================] - 8s 39ms/step - loss: 0.8412 - accuracy: 0.7124 - val_loss: 1.3872 - val_accuracy: 0.5467
Epoch 5/10
213/213 [==============================] - 8s 39ms/step - loss: 0.7163 - accuracy: 0.7584 - val_loss: 1.4764 - val_accuracy: 0.5437
Epoch 6/10
213/213 [==============================] - 8s 39ms/step - loss: 0.5905 - accuracy: 0.8082 - val_loss: 1.5786 - val_accuracy: 0.5325
Epoch 7/10
213/213 [==============================] - 8s 39ms/step - loss: 0.4

## Bankings77 Dataset Training

In [14]:
data_path = './Data/bankings_77'
texts, labels = load_data(data_path)
processed_text, labels = get_preprocessed_data(texts, labels)
X, y = split_data(processed_text, labels, validation_size, test_size)
X, y = tokenize_data(X, y, labels, vocab_size, oov_token, padding_size, padding_type)

In [18]:
hidden_layer_size = 128
num_labels = len(set(labels))
loss = 'sparse_categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy']
epochs = 60
batch_size = 128
weight_decays_l1 = [0, 0.001, 0.01, 0.1]
weight_decays_l2 = [0.001, 0.01, 0.1]
dropout_rates = [0.1, 0.2, 0.3]
dropconnect_rates = [0.1, 0.2, 0.3]
dataset = os.path.basename(data_path)

In [19]:
for weight_decay_l1 in weight_decays_l1:
    print(f'Training RNN on {dataset} with {weight_decay_l1} L1 Weight Decay...')
    weight_decay = l1(weight_decay_l1) if weight_decay_l1 > 0 else None  
    rnn_model = get_model(vocab_size, hidden_layer_size, 'relu', num_labels, weight_decay, 0, 0, 'rnn')
    run_model(X, y, rnn_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'rnn_l1_weight_decay_{weight_decay_l1}_{dataset}')

Training RNN on bankings_77 with 0 L1 Weight Decay...


2022-05-06 22:24:55.213452: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 22:24:55.223665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 22:24:55.224348: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 22:24:55.226590: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/60
14/43 [========>.....................] - ETA: 9s - loss: 4.3402 - accuracy: 0.0162

KeyboardInterrupt: 

In [24]:
for weight_decay_l2 in weight_decays_l2:
    print(f'Training RNN on {dataset} with {weight_decay_l2} L2 Weight Decay...')
    weight_decay = l2(weight_decay_l2) if weight_decay_l2 > 0 else None
    rnn_model = get_model(vocab_size, hidden_layer_size, 'relu', num_labels, weight_decay, 0, 0, 'rnn')
    run_model(X, y, rnn_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'rnn_l2_weight_decay_{weight_decay_l2}_{dataset}')

Training RNN on bankings_77 with 0.001 L2 Weight Decay...
Epoch 1/60
43/43 [==============================] - 32s 310ms/step - loss: 4.5050 - accuracy: 0.0259 - val_loss: 4.4013 - val_accuracy: 0.0250
Epoch 2/60
43/43 [==============================] - 13s 313ms/step - loss: 56.3256 - accuracy: 0.0890 - val_loss: 3.7266 - val_accuracy: 0.1521
Epoch 3/60
43/43 [==============================] - 13s 294ms/step - loss: 3.6634 - accuracy: 0.1450 - val_loss: 3.3398 - val_accuracy: 0.1802
Epoch 4/60
43/43 [==============================] - 13s 294ms/step - loss: 11.0815 - accuracy: 0.2825 - val_loss: 3.5568 - val_accuracy: 0.1875
Epoch 5/60
43/43 [==============================] - 14s 322ms/step - loss: 4.8538 - accuracy: 0.2290 - val_loss: 3.2582 - val_accuracy: 0.2542
Epoch 6/60
43/43 [==============================] - 13s 308ms/step - loss: 2.8022 - accuracy: 0.3233 - val_loss: 2.6440 - val_accuracy: 0.3208
Epoch 7/60
43/43 [==============================] - 13s 311ms/step - loss: 2.1953 

43/43 [==============================] - 14s 326ms/step - loss: 0.4184 - accuracy: 0.9529 - val_loss: 1.6913 - val_accuracy: 0.7281
Epoch 54/60
43/43 [==============================] - 14s 317ms/step - loss: 0.4495 - accuracy: 0.9452 - val_loss: 1.6995 - val_accuracy: 0.7333
Epoch 55/60
43/43 [==============================] - 14s 314ms/step - loss: 0.4229 - accuracy: 0.9539 - val_loss: 1.5378 - val_accuracy: 0.7250
Epoch 56/60
43/43 [==============================] - 14s 316ms/step - loss: 0.4159 - accuracy: 0.9561 - val_loss: 1.6369 - val_accuracy: 0.7219
Epoch 57/60
43/43 [==============================] - 14s 320ms/step - loss: 0.3817 - accuracy: 0.9619 - val_loss: 1.6324 - val_accuracy: 0.7271
Epoch 58/60
43/43 [==============================] - 13s 305ms/step - loss: 0.4145 - accuracy: 0.9517 - val_loss: 1.6154 - val_accuracy: 0.7281
Epoch 59/60
43/43 [==============================] - 13s 313ms/step - loss: 0.3831 - accuracy: 0.9608 - val_loss: 1.6326 - val_accuracy: 0.7250
Epoc

Epoch 49/60
43/43 [==============================] - 14s 316ms/step - loss: 1.4867 - accuracy: 0.9103 - val_loss: 2.4271 - val_accuracy: 0.6917
Epoch 50/60
43/43 [==============================] - 14s 330ms/step - loss: 1.4799 - accuracy: 0.9092 - val_loss: 2.5061 - val_accuracy: 0.6854
Epoch 51/60
43/43 [==============================] - 13s 314ms/step - loss: 1.4985 - accuracy: 0.9017 - val_loss: 2.5155 - val_accuracy: 0.6729
Epoch 52/60
43/43 [==============================] - 13s 305ms/step - loss: 1.4725 - accuracy: 0.9088 - val_loss: 2.4539 - val_accuracy: 0.6854
Epoch 53/60
43/43 [==============================] - 13s 296ms/step - loss: 1.4554 - accuracy: 0.9140 - val_loss: 2.5095 - val_accuracy: 0.6802
Epoch 54/60
43/43 [==============================] - 13s 300ms/step - loss: 1.4507 - accuracy: 0.9171 - val_loss: 2.4985 - val_accuracy: 0.6896
Epoch 55/60
43/43 [==============================] - 15s 344ms/step - loss: 1.4545 - accuracy: 0.9129 - val_loss: 2.4996 - val_accuracy:

In [25]:
for dropout_rate in dropout_rates:
    print(f'Training RNN on {dataset} with {dropout_rate} Dropout...')
    rnn_model = get_model(vocab_size, hidden_layer_size, 'relu', num_labels, None, dropout_rate, 0, 'rnn')
    run_model(X, y, rnn_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'rnn_dropout_{dropout_rate}_{dataset}')

Training RNN on bankings_77 with 0.1 Dropout...
Epoch 1/60
43/43 [==============================] - 18s 400ms/step - loss: 671844.6250 - accuracy: 0.0300 - val_loss: 4.2918 - val_accuracy: 0.0417
Epoch 2/60
43/43 [==============================] - 17s 398ms/step - loss: 4.1722 - accuracy: 0.0768 - val_loss: 4.0380 - val_accuracy: 0.1365
Epoch 3/60
43/43 [==============================] - 19s 446ms/step - loss: 3.6485 - accuracy: 0.1857 - val_loss: 3.3076 - val_accuracy: 0.1896
Epoch 4/60
43/43 [==============================] - 18s 411ms/step - loss: 2.9648 - accuracy: 0.3243 - val_loss: 2.7440 - val_accuracy: 0.3240
Epoch 5/60
43/43 [==============================] - 17s 404ms/step - loss: 2.2701 - accuracy: 0.4623 - val_loss: 2.2696 - val_accuracy: 0.4073
Epoch 6/60
43/43 [==============================] - 18s 413ms/step - loss: 1.8129 - accuracy: 0.5684 - val_loss: 1.9833 - val_accuracy: 0.4844
Epoch 7/60
43/43 [==============================] - 19s 436ms/step - loss: 1.4700 - accur

43/43 [==============================] - 17s 386ms/step - loss: 0.1291 - accuracy: 0.9711 - val_loss: 1.2627 - val_accuracy: 0.7500
Epoch 54/60
43/43 [==============================] - 17s 387ms/step - loss: 0.1255 - accuracy: 0.9697 - val_loss: 1.2647 - val_accuracy: 0.7500
Epoch 55/60
43/43 [==============================] - 17s 406ms/step - loss: 0.1152 - accuracy: 0.9721 - val_loss: 1.2722 - val_accuracy: 0.7437
Epoch 56/60
43/43 [==============================] - 16s 381ms/step - loss: 0.1088 - accuracy: 0.9733 - val_loss: 1.3208 - val_accuracy: 0.7375
Epoch 57/60
43/43 [==============================] - 16s 382ms/step - loss: 0.1010 - accuracy: 0.9767 - val_loss: 1.2978 - val_accuracy: 0.7417
Epoch 58/60
43/43 [==============================] - 17s 404ms/step - loss: 0.1044 - accuracy: 0.9728 - val_loss: 1.3321 - val_accuracy: 0.7437
Epoch 59/60
43/43 [==============================] - 16s 382ms/step - loss: 0.1153 - accuracy: 0.9699 - val_loss: 1.2766 - val_accuracy: 0.7448
Epoc

In [26]:
for dropconnect_rate in dropconnect_rates:
    print(f'Training RNN on {dataset} with {dropconnect_rate} Dropconnect...')
    rnn_model = get_model(vocab_size, hidden_layer_size, 'relu', num_labels, None, 0, dropconnect_rate, 'rnn')
    run_model(X, y, rnn_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'rnn_dropconnect_{dropconnect_rate}_{dataset}')

Training RNN on bankings_77 with 0.1 Dropconnect...
Epoch 1/60
43/43 [==============================] - 15s 312ms/step - loss: 2360.7861 - accuracy: 0.0276 - val_loss: 4.2971 - val_accuracy: 0.0552
Epoch 2/60
43/43 [==============================] - 14s 329ms/step - loss: 4.1517 - accuracy: 0.0961 - val_loss: 3.9467 - val_accuracy: 0.1000
Epoch 3/60
43/43 [==============================] - 13s 308ms/step - loss: 3.3918 - accuracy: 0.1993 - val_loss: 2.9382 - val_accuracy: 0.2969
Epoch 4/60
43/43 [==============================] - 13s 309ms/step - loss: 2.3849 - accuracy: 0.4279 - val_loss: 2.2538 - val_accuracy: 0.4271
Epoch 5/60
43/43 [==============================] - 13s 299ms/step - loss: 1.7907 - accuracy: 0.5629 - val_loss: 1.8454 - val_accuracy: 0.5146
Epoch 6/60
43/43 [==============================] - 13s 307ms/step - loss: 1.3660 - accuracy: 0.6618 - val_loss: 1.6247 - val_accuracy: 0.5635
Epoch 7/60
43/43 [==============================] - 14s 333ms/step - loss: 1.0818 - acc

43/43 [==============================] - 14s 322ms/step - loss: 0.2941 - accuracy: 0.9395 - val_loss: 1.3582 - val_accuracy: 0.6583
Epoch 54/60
43/43 [==============================] - 13s 314ms/step - loss: 0.2712 - accuracy: 0.9467 - val_loss: 1.3682 - val_accuracy: 0.6667
Epoch 55/60
43/43 [==============================] - 13s 305ms/step - loss: 0.2524 - accuracy: 0.9513 - val_loss: 1.3557 - val_accuracy: 0.6656
Epoch 56/60
43/43 [==============================] - 14s 331ms/step - loss: 0.2326 - accuracy: 0.9579 - val_loss: 1.3640 - val_accuracy: 0.6698
Epoch 57/60
43/43 [==============================] - 14s 318ms/step - loss: 0.2171 - accuracy: 0.9597 - val_loss: 1.3691 - val_accuracy: 0.6667
Epoch 58/60
43/43 [==============================] - 14s 316ms/step - loss: 0.2026 - accuracy: 0.9631 - val_loss: 1.3658 - val_accuracy: 0.6750
Epoch 59/60
43/43 [==============================] - 14s 324ms/step - loss: 0.1902 - accuracy: 0.9676 - val_loss: 1.3789 - val_accuracy: 0.6802
Epoc

KeyboardInterrupt: 

In [16]:
hidden_layer_size = 64
num_labels = len(set(labels))
loss = 'sparse_categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy']
epochs = 80
batch_size = 64
weight_decays_l1 = [0, 0.001, 0.01, 0.1]
weight_decays_l2 = [0.001, 0.01, 0.1]
dropout_rates = [0.1, 0.2, 0.3]
dropconnect_rates = [0.1, 0.2, 0.3]
dataset = os.path.basename(data_path)

In [23]:
for weight_decay_l1 in weight_decays_l1:
    print(f'Training LSTM on {dataset} with {weight_decay_l1} L1 Weight Decay...')
    weight_decay = l1(weight_decay_l1) if weight_decay_l1 > 0 else None
    lstm_model = get_model(vocab_size, hidden_layer_size, 'tanh', num_labels, weight_decay, 0, 0, 'lstm')
    run_model(X, y, lstm_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'lstm_l1_weight_decay_{weight_decay_l1}_{dataset}')

Training LSTM on bankings_77 with 0 L1 Weight Decay...
Epoch 1/80
85/85 [==============================] - 5s 34ms/step - loss: 4.3093 - accuracy: 0.0355 - val_loss: 4.1943 - val_accuracy: 0.0740
Epoch 2/80
85/85 [==============================] - 2s 27ms/step - loss: 3.6336 - accuracy: 0.1325 - val_loss: 3.0629 - val_accuracy: 0.1604
Epoch 3/80
85/85 [==============================] - 2s 27ms/step - loss: 2.5033 - accuracy: 0.3094 - val_loss: 2.3080 - val_accuracy: 0.3521
Epoch 4/80
85/85 [==============================] - 2s 27ms/step - loss: 1.7912 - accuracy: 0.5079 - val_loss: 1.8381 - val_accuracy: 0.4802
Epoch 5/80
85/85 [==============================] - 2s 27ms/step - loss: 1.3253 - accuracy: 0.6439 - val_loss: 1.5116 - val_accuracy: 0.5958
Epoch 6/80
85/85 [==============================] - 2s 27ms/step - loss: 1.0639 - accuracy: 0.7210 - val_loss: 1.5377 - val_accuracy: 0.5875
Epoch 7/80
85/85 [==============================] - 2s 27ms/step - loss: 0.8670 - accuracy: 0.7772 

Epoch 35/80
85/85 [==============================] - 3s 30ms/step - loss: 0.3377 - accuracy: 0.9436 - val_loss: 1.5112 - val_accuracy: 0.7073
Epoch 36/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3262 - accuracy: 0.9417 - val_loss: 1.4936 - val_accuracy: 0.7208
Epoch 37/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3093 - accuracy: 0.9480 - val_loss: 1.5018 - val_accuracy: 0.7292
Epoch 38/80
85/85 [==============================] - 2s 27ms/step - loss: 0.3052 - accuracy: 0.9467 - val_loss: 1.5124 - val_accuracy: 0.7312
Epoch 39/80
85/85 [==============================] - 2s 28ms/step - loss: 0.2903 - accuracy: 0.9553 - val_loss: 1.5327 - val_accuracy: 0.7375
Epoch 40/80
85/85 [==============================] - 2s 28ms/step - loss: 0.2747 - accuracy: 0.9561 - val_loss: 1.5228 - val_accuracy: 0.7333
Epoch 41/80
85/85 [==============================] - 2s 28ms/step - loss: 0.2771 - accuracy: 0.9515 - val_loss: 1.5250 - val_accuracy: 0.7333
Epoch 

Epoch 12/80
85/85 [==============================] - 2s 27ms/step - loss: 4.3620 - accuracy: 0.0164 - val_loss: 4.3744 - val_accuracy: 0.0188
Epoch 13/80
85/85 [==============================] - 2s 27ms/step - loss: 4.3619 - accuracy: 0.0156 - val_loss: 4.3765 - val_accuracy: 0.0188
Epoch 14/80
85/85 [==============================] - 2s 27ms/step - loss: 4.3623 - accuracy: 0.0165 - val_loss: 4.3749 - val_accuracy: 0.0188
Epoch 15/80
85/85 [==============================] - 2s 27ms/step - loss: 4.3615 - accuracy: 0.0175 - val_loss: 4.3779 - val_accuracy: 0.0177
Epoch 16/80
85/85 [==============================] - 2s 27ms/step - loss: 4.3616 - accuracy: 0.0175 - val_loss: 4.3773 - val_accuracy: 0.0188
Epoch 17/80
85/85 [==============================] - 2s 29ms/step - loss: 4.3617 - accuracy: 0.0178 - val_loss: 4.3758 - val_accuracy: 0.0188
Epoch 18/80
85/85 [==============================] - 2s 27ms/step - loss: 4.3613 - accuracy: 0.0162 - val_loss: 4.3768 - val_accuracy: 0.0188
Epoch 

Epoch 70/80
85/85 [==============================] - 2s 29ms/step - loss: 4.3596 - accuracy: 0.0167 - val_loss: 4.3751 - val_accuracy: 0.0177
Epoch 71/80
85/85 [==============================] - 2s 27ms/step - loss: 4.3595 - accuracy: 0.0156 - val_loss: 4.3746 - val_accuracy: 0.0188
Epoch 72/80
85/85 [==============================] - 2s 27ms/step - loss: 4.1959 - accuracy: 0.0230 - val_loss: 3.8996 - val_accuracy: 0.0323
Epoch 73/80
85/85 [==============================] - 2s 27ms/step - loss: 3.7327 - accuracy: 0.0511 - val_loss: 3.7486 - val_accuracy: 0.0510
Epoch 74/80
85/85 [==============================] - 2s 27ms/step - loss: 3.5625 - accuracy: 0.0682 - val_loss: 3.5773 - val_accuracy: 0.0531
Epoch 75/80
85/85 [==============================] - 2s 27ms/step - loss: 3.4068 - accuracy: 0.0842 - val_loss: 3.4724 - val_accuracy: 0.0844
Epoch 76/80
85/85 [==============================] - 2s 27ms/step - loss: 3.5807 - accuracy: 0.0824 - val_loss: 3.5716 - val_accuracy: 0.0885
Epoch 

Epoch 47/80
85/85 [==============================] - 2s 27ms/step - loss: 4.7139 - accuracy: 0.0173 - val_loss: 4.7254 - val_accuracy: 0.0188
Epoch 48/80
85/85 [==============================] - 2s 27ms/step - loss: 4.7141 - accuracy: 0.0178 - val_loss: 4.7276 - val_accuracy: 0.0188
Epoch 49/80
85/85 [==============================] - 2s 27ms/step - loss: 4.7137 - accuracy: 0.0154 - val_loss: 4.7288 - val_accuracy: 0.0188
Epoch 50/80
85/85 [==============================] - 2s 27ms/step - loss: 4.7141 - accuracy: 0.0162 - val_loss: 4.7306 - val_accuracy: 0.0188
Epoch 51/80
85/85 [==============================] - 2s 26ms/step - loss: 4.7137 - accuracy: 0.0154 - val_loss: 4.7320 - val_accuracy: 0.0188
Epoch 52/80
85/85 [==============================] - 2s 26ms/step - loss: 4.7143 - accuracy: 0.0178 - val_loss: 4.7251 - val_accuracy: 0.0188
Epoch 53/80
85/85 [==============================] - 2s 26ms/step - loss: 4.7136 - accuracy: 0.0178 - val_loss: 4.7262 - val_accuracy: 0.0188
Epoch 

In [20]:
for weight_decay_l2 in weight_decays_l2:
    print(f'Training LSTM on {dataset} with {weight_decay_l2} L2 Weight Decay...')
    weight_decay = l2(weight_decay_l2) if weight_decay_l2 > 0 else None
    lstm_model = get_model(vocab_size, hidden_layer_size, 'tanh', num_labels, weight_decay, 0, 0, 'lstm')
    run_model(X, y, lstm_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'lstm_l2_weight_decay_{weight_decay_l2}_{dataset}')

Training LSTM on bankings_77 with 0.001 L2 Weight Decay...
Epoch 1/80
85/85 [==============================] - 5s 35ms/step - loss: 4.4231 - accuracy: 0.0199 - val_loss: 4.3243 - val_accuracy: 0.0167
Epoch 2/80
85/85 [==============================] - 2s 28ms/step - loss: 3.9887 - accuracy: 0.0711 - val_loss: 3.4632 - val_accuracy: 0.1771
Epoch 3/80
85/85 [==============================] - 2s 28ms/step - loss: 2.7164 - accuracy: 0.3195 - val_loss: 2.3113 - val_accuracy: 0.4104
Epoch 4/80
85/85 [==============================] - 2s 28ms/step - loss: 1.8543 - accuracy: 0.5281 - val_loss: 1.7934 - val_accuracy: 0.5042
Epoch 5/80
85/85 [==============================] - 2s 28ms/step - loss: 1.4270 - accuracy: 0.6307 - val_loss: 1.5315 - val_accuracy: 0.6073
Epoch 6/80
85/85 [==============================] - 2s 28ms/step - loss: 1.0851 - accuracy: 0.7303 - val_loss: 1.3690 - val_accuracy: 0.6354
Epoch 7/80
85/85 [==============================] - 2s 28ms/step - loss: 0.9035 - accuracy: 0.7

Epoch 35/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3867 - accuracy: 0.9204 - val_loss: 1.4463 - val_accuracy: 0.6875
Epoch 36/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3873 - accuracy: 0.9140 - val_loss: 1.4087 - val_accuracy: 0.6865
Epoch 37/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3699 - accuracy: 0.9191 - val_loss: 1.4357 - val_accuracy: 0.6979
Epoch 38/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3552 - accuracy: 0.9256 - val_loss: 1.4561 - val_accuracy: 0.7021
Epoch 39/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3416 - accuracy: 0.9296 - val_loss: 1.4697 - val_accuracy: 0.7021
Epoch 40/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3311 - accuracy: 0.9324 - val_loss: 1.4612 - val_accuracy: 0.7083
Epoch 41/80
85/85 [==============================] - 3s 30ms/step - loss: 0.3347 - accuracy: 0.9303 - val_loss: 1.4845 - val_accuracy: 0.6990
Epoch 

Epoch 12/80
85/85 [==============================] - 2s 28ms/step - loss: 3.1440 - accuracy: 0.1250 - val_loss: 3.2542 - val_accuracy: 0.1208
Epoch 13/80
85/85 [==============================] - 2s 28ms/step - loss: 2.9888 - accuracy: 0.1528 - val_loss: 3.1459 - val_accuracy: 0.1333
Epoch 14/80
85/85 [==============================] - 2s 28ms/step - loss: 2.8278 - accuracy: 0.1895 - val_loss: 3.0194 - val_accuracy: 0.1667
Epoch 15/80
85/85 [==============================] - 2s 28ms/step - loss: 2.8502 - accuracy: 0.1901 - val_loss: 2.9045 - val_accuracy: 0.1802
Epoch 16/80
85/85 [==============================] - 2s 28ms/step - loss: 2.6527 - accuracy: 0.2184 - val_loss: 2.8130 - val_accuracy: 0.2219
Epoch 17/80
85/85 [==============================] - 2s 28ms/step - loss: 2.4896 - accuracy: 0.2647 - val_loss: 2.7295 - val_accuracy: 0.2229
Epoch 18/80
85/85 [==============================] - 2s 28ms/step - loss: 2.5283 - accuracy: 0.2562 - val_loss: 2.7087 - val_accuracy: 0.2490
Epoch 

Epoch 70/80
85/85 [==============================] - 2s 28ms/step - loss: 0.5493 - accuracy: 0.8741 - val_loss: 1.6471 - val_accuracy: 0.6646
Epoch 71/80
85/85 [==============================] - 2s 28ms/step - loss: 0.5401 - accuracy: 0.8733 - val_loss: 1.7097 - val_accuracy: 0.6542
Epoch 72/80
85/85 [==============================] - 2s 28ms/step - loss: 0.5330 - accuracy: 0.8743 - val_loss: 1.6671 - val_accuracy: 0.6583
Epoch 73/80
85/85 [==============================] - 2s 28ms/step - loss: 0.5216 - accuracy: 0.8796 - val_loss: 1.7202 - val_accuracy: 0.6573
Epoch 74/80
85/85 [==============================] - 2s 28ms/step - loss: 0.5326 - accuracy: 0.8781 - val_loss: 1.6989 - val_accuracy: 0.6583
Epoch 75/80
85/85 [==============================] - 2s 28ms/step - loss: 0.5187 - accuracy: 0.8816 - val_loss: 1.7370 - val_accuracy: 0.6615
Epoch 76/80
85/85 [==============================] - 2s 28ms/step - loss: 0.5007 - accuracy: 0.8827 - val_loss: 1.7135 - val_accuracy: 0.6625
Epoch 

In [22]:
for dropout_rate in dropout_rates:
    print(f'Training LSTM on {dataset} with {dropout_rate} Dropout...')
    lstm_model = get_model(vocab_size, hidden_layer_size, 'tanh', num_labels, None, dropout_rate, 0, 'lstm')
    run_model(X, y, lstm_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'lstm_dropout_{dropout_rate}_{dataset}')

Training LSTM on bankings_77 with 0.1 Dropout...
Epoch 1/80
85/85 [==============================] - 6s 41ms/step - loss: 4.3120 - accuracy: 0.0406 - val_loss: 4.1916 - val_accuracy: 0.0615
Epoch 2/80
85/85 [==============================] - 2s 28ms/step - loss: 3.5701 - accuracy: 0.1399 - val_loss: 2.8909 - val_accuracy: 0.2490
Epoch 3/80
85/85 [==============================] - 2s 28ms/step - loss: 2.3410 - accuracy: 0.3809 - val_loss: 2.0539 - val_accuracy: 0.4448
Epoch 4/80
85/85 [==============================] - 2s 28ms/step - loss: 1.5982 - accuracy: 0.5993 - val_loss: 1.5951 - val_accuracy: 0.5573
Epoch 5/80
85/85 [==============================] - 2s 28ms/step - loss: 1.1564 - accuracy: 0.7070 - val_loss: 1.3192 - val_accuracy: 0.6552
Epoch 6/80
85/85 [==============================] - 3s 30ms/step - loss: 0.8875 - accuracy: 0.7754 - val_loss: 1.1696 - val_accuracy: 0.6938
Epoch 7/80
85/85 [==============================] - 2s 29ms/step - loss: 0.7167 - accuracy: 0.8156 - val_

Epoch 35/80
85/85 [==============================] - 2s 28ms/step - loss: 0.1302 - accuracy: 0.9640 - val_loss: 1.1583 - val_accuracy: 0.7667
Epoch 36/80
85/85 [==============================] - 2s 28ms/step - loss: 0.1425 - accuracy: 0.9570 - val_loss: 1.1564 - val_accuracy: 0.7667
Epoch 37/80
85/85 [==============================] - 2s 28ms/step - loss: 0.1282 - accuracy: 0.9612 - val_loss: 1.1549 - val_accuracy: 0.7677
Epoch 38/80
85/85 [==============================] - 2s 28ms/step - loss: 0.1300 - accuracy: 0.9625 - val_loss: 1.1916 - val_accuracy: 0.7625
Epoch 39/80
85/85 [==============================] - 2s 28ms/step - loss: 0.2207 - accuracy: 0.9375 - val_loss: 1.2374 - val_accuracy: 0.7552
Epoch 40/80
85/85 [==============================] - 2s 28ms/step - loss: 0.1506 - accuracy: 0.9550 - val_loss: 1.1850 - val_accuracy: 0.7635
Epoch 41/80
85/85 [==============================] - 2s 28ms/step - loss: 0.1245 - accuracy: 0.9636 - val_loss: 1.1588 - val_accuracy: 0.7719
Epoch 

Epoch 12/80
85/85 [==============================] - 2s 28ms/step - loss: 0.4403 - accuracy: 0.8840 - val_loss: 0.9252 - val_accuracy: 0.7604
Epoch 13/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3998 - accuracy: 0.8914 - val_loss: 0.9070 - val_accuracy: 0.7719
Epoch 14/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3797 - accuracy: 0.8961 - val_loss: 0.9034 - val_accuracy: 0.7719
Epoch 15/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3427 - accuracy: 0.9070 - val_loss: 0.9268 - val_accuracy: 0.7594
Epoch 16/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3136 - accuracy: 0.9125 - val_loss: 0.9103 - val_accuracy: 0.7823
Epoch 17/80
85/85 [==============================] - 2s 28ms/step - loss: 0.3056 - accuracy: 0.9147 - val_loss: 0.9166 - val_accuracy: 0.7750
Epoch 18/80
85/85 [==============================] - 2s 28ms/step - loss: 0.2895 - accuracy: 0.9180 - val_loss: 0.9421 - val_accuracy: 0.7688
Epoch 

Epoch 70/80
85/85 [==============================] - 2s 28ms/step - loss: 0.0677 - accuracy: 0.9787 - val_loss: 1.2824 - val_accuracy: 0.7906
Epoch 71/80
85/85 [==============================] - 2s 28ms/step - loss: 0.0653 - accuracy: 0.9796 - val_loss: 1.2778 - val_accuracy: 0.7781
Epoch 72/80
85/85 [==============================] - 2s 28ms/step - loss: 0.0675 - accuracy: 0.9779 - val_loss: 1.2320 - val_accuracy: 0.7875
Epoch 73/80
85/85 [==============================] - 2s 28ms/step - loss: 0.0653 - accuracy: 0.9809 - val_loss: 1.2711 - val_accuracy: 0.7927
Epoch 74/80
85/85 [==============================] - 2s 28ms/step - loss: 0.0746 - accuracy: 0.9772 - val_loss: 1.2320 - val_accuracy: 0.7677
Epoch 75/80
85/85 [==============================] - 2s 28ms/step - loss: 0.0623 - accuracy: 0.9820 - val_loss: 1.2332 - val_accuracy: 0.7948
Epoch 76/80
85/85 [==============================] - 2s 28ms/step - loss: 0.0534 - accuracy: 0.9844 - val_loss: 1.2298 - val_accuracy: 0.7833
Epoch 

In [17]:
for dropconnect_rate in dropconnect_rates:
    print(f'Training LSTM on {dataset} with {dropconnect_rate} Dropconnect...')
    lstm_model = get_model(vocab_size, hidden_layer_size, 'tanh', num_labels, None, 0, dropconnect_rate, 'lstm')
    run_model(X, y, lstm_model, loss, optimizer, metrics, epochs, batch_size, save_path, f'lstm_dropconnect_{dropconnect_rate}_{dataset}')

Training LSTM on bankings_77 with 0.1 Dropconnect...


2022-05-06 23:21:16.337578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 23:21:16.349089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 23:21:16.349739: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 23:21:16.351246: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/80


2022-05-06 23:21:20.246555: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


85/85 [==============================] - 6s 35ms/step - loss: 4.3100 - accuracy: 0.0250 - val_loss: 4.1694 - val_accuracy: 0.0177
Epoch 2/80
85/85 [==============================] - 2s 27ms/step - loss: 3.6417 - accuracy: 0.1200 - val_loss: 3.1335 - val_accuracy: 0.1917
Epoch 3/80
85/85 [==============================] - 2s 27ms/step - loss: 2.4603 - accuracy: 0.3590 - val_loss: 2.1155 - val_accuracy: 0.4365
Epoch 4/80
85/85 [==============================] - 2s 27ms/step - loss: 1.6205 - accuracy: 0.5847 - val_loss: 1.6631 - val_accuracy: 0.5604
Epoch 5/80
85/85 [==============================] - 2s 27ms/step - loss: 1.1725 - accuracy: 0.7017 - val_loss: 1.4077 - val_accuracy: 0.6333
Epoch 6/80
85/85 [==============================] - 2s 27ms/step - loss: 0.9226 - accuracy: 0.7678 - val_loss: 1.3324 - val_accuracy: 0.6396
Epoch 7/80
85/85 [==============================] - 2s 27ms/step - loss: 0.7350 - accuracy: 0.8186 - val_loss: 1.2324 - val_accuracy: 0.6750
Epoch 8/80
85/85 [======

Epoch 59/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0230 - accuracy: 0.9941 - val_loss: 1.8261 - val_accuracy: 0.7479
Epoch 60/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0282 - accuracy: 0.9925 - val_loss: 1.8098 - val_accuracy: 0.7437
Epoch 61/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0235 - accuracy: 0.9923 - val_loss: 1.8317 - val_accuracy: 0.7479
Epoch 62/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0217 - accuracy: 0.9943 - val_loss: 1.8512 - val_accuracy: 0.7500
Epoch 63/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0242 - accuracy: 0.9936 - val_loss: 1.8860 - val_accuracy: 0.7333
Epoch 64/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0330 - accuracy: 0.9914 - val_loss: 1.9802 - val_accuracy: 0.7198
Epoch 65/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0543 - accuracy: 0.9833 - val_loss: 2.0375 - val_accuracy: 0.7177
Epoch 

Epoch 36/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0727 - accuracy: 0.9833 - val_loss: 1.3596 - val_accuracy: 0.7500
Epoch 37/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0661 - accuracy: 0.9836 - val_loss: 1.3576 - val_accuracy: 0.7531
Epoch 38/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0609 - accuracy: 0.9853 - val_loss: 1.4094 - val_accuracy: 0.7479
Epoch 39/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0565 - accuracy: 0.9881 - val_loss: 1.4318 - val_accuracy: 0.7510
Epoch 40/80
85/85 [==============================] - 2s 28ms/step - loss: 0.0514 - accuracy: 0.9888 - val_loss: 1.3988 - val_accuracy: 0.7615
Epoch 41/80
85/85 [==============================] - 2s 28ms/step - loss: 0.0533 - accuracy: 0.9869 - val_loss: 1.4403 - val_accuracy: 0.7479
Epoch 42/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0510 - accuracy: 0.9879 - val_loss: 1.4863 - val_accuracy: 0.7448
Epoch 

Epoch 13/80
85/85 [==============================] - 2s 27ms/step - loss: 0.2835 - accuracy: 0.9320 - val_loss: 1.0881 - val_accuracy: 0.7344
Epoch 14/80
85/85 [==============================] - 2s 27ms/step - loss: 0.2548 - accuracy: 0.9375 - val_loss: 1.0876 - val_accuracy: 0.7521
Epoch 15/80
85/85 [==============================] - 2s 27ms/step - loss: 0.2295 - accuracy: 0.9439 - val_loss: 1.0735 - val_accuracy: 0.7469
Epoch 16/80
85/85 [==============================] - 2s 27ms/step - loss: 0.2107 - accuracy: 0.9500 - val_loss: 1.2037 - val_accuracy: 0.7396
Epoch 17/80
85/85 [==============================] - 2s 27ms/step - loss: 0.2035 - accuracy: 0.9502 - val_loss: 1.0688 - val_accuracy: 0.7552
Epoch 18/80
85/85 [==============================] - 2s 27ms/step - loss: 0.1808 - accuracy: 0.9592 - val_loss: 1.1362 - val_accuracy: 0.7563
Epoch 19/80
85/85 [==============================] - 2s 27ms/step - loss: 0.1663 - accuracy: 0.9623 - val_loss: 1.1935 - val_accuracy: 0.7396
Epoch 

Epoch 71/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0179 - accuracy: 0.9950 - val_loss: 1.9338 - val_accuracy: 0.7552
Epoch 72/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0172 - accuracy: 0.9958 - val_loss: 1.9022 - val_accuracy: 0.7583
Epoch 73/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0207 - accuracy: 0.9936 - val_loss: 1.9542 - val_accuracy: 0.7510
Epoch 74/80
85/85 [==============================] - 3s 30ms/step - loss: 0.0170 - accuracy: 0.9947 - val_loss: 2.0192 - val_accuracy: 0.7427
Epoch 75/80
85/85 [==============================] - 2s 28ms/step - loss: 0.0169 - accuracy: 0.9947 - val_loss: 2.0089 - val_accuracy: 0.7521
Epoch 76/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0173 - accuracy: 0.9943 - val_loss: 1.9529 - val_accuracy: 0.7510
Epoch 77/80
85/85 [==============================] - 2s 27ms/step - loss: 0.0166 - accuracy: 0.9952 - val_loss: 1.9472 - val_accuracy: 0.7563
Epoch 

### References
The references and resources were used for developing the code.
- https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17
- https://towardsdatascience.com/multi-class-text-classification-with-lstm-using-tensorflow-2-0-d88627c10a35
- https://huggingface.co/docs/datasets/index